In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from textblob import TextBlob
from tabulate import tabulate
from sklearn.ensemble import IsolationForest

Matplotlib is building the font cache; this may take a moment.


In [2]:
listings_NY = pd.read_csv("listings_newyork.csv")
reviews_NY = pd.read_csv("reviews_newyork.csv")

In [3]:
print(listings_NY.columns)
print(reviews_NY.columns)

Index(['listing_id', 'listing_url', 'scrape_id', 'last_scraped', 'source',
       'name', 'description', 'neighborhood_overview', 'picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_n

In [4]:
# Merge Albany listings and reviews
Combined_Albany = pd.merge(
    listings_NY,
    reviews_NY,
    on='listing_id',   # common column in both DataFrames
    how='left'         # keep all listings, match reviews if available
)

# Save merged dataset
Combined_Albany.to_csv("Albany.csv", index=False)

In [5]:
Combined_Albany.isnull().sum()

listing_id         0
listing_url        0
scrape_id          0
last_scraped       0
source             0
                ... 
id               325
date             325
reviewer_id      325
reviewer_name    325
comments         333
Length: 84, dtype: int64

In [6]:
Combined_Albany['comments'] = Combined_Albany['comments'].fillna('No review')
Combined_Albany['reviewer_name'] = Combined_Albany['reviewer_name'].fillna('Unknown')
Combined_Albany['reviewer_id'] = Combined_Albany['reviewer_id'].fillna('0')
Combined_Albany['date'] = Combined_Albany['date'].fillna('0')
Combined_Albany['id'] = Combined_Albany['id'].fillna('0')

In [7]:
Combined_Albany.isnull().sum()

listing_id       0
listing_url      0
scrape_id        0
last_scraped     0
source           0
                ..
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
Length: 84, dtype: int64

In [8]:
# Create summary table
summary = pd.DataFrame({
    'Column': Combined_Albany.columns,
    'Total_Rows': len(Combined_Albany),
    'Unique_Values': [Combined_Albany[col].nunique() for col in Combined_Albany.columns],
    'Duplicate_Count': [Combined_Albany[col].duplicated().sum() for col in Combined_Albany.columns],
    'Duplicate_Percentage': [round((Combined_Albany[col].duplicated().sum() / len(Combined_Albany)) * 100, 2) for col in Combined_Albany.columns]
})

# Display full table without truncation
pd.set_option('display.max_rows', None)
print(summary)

                                          Column  Total_Rows  Unique_Values  \
0                                     listing_id       18899            431   
1                                    listing_url       18899            447   
2                                      scrape_id       18899              1   
3                                   last_scraped       18899              1   
4                                         source       18899              2   
5                                           name       18899            447   
6                                    description       18899            395   
7                          neighborhood_overview       18899            157   
8                                    picture_url       18899            441   
9                                        host_id       18899            193   
10                                      host_url       18899            193   
11                                     host_name    

In [9]:
# Load datasets
listings_NL = pd.read_csv("listings_netherlands.csv")
reviews_NL = pd.read_csv(
    "reviews_netherlands.csv",
    engine="python",      # more tolerant parser
    quotechar='"',        # handle quoted text
    escapechar='\\',      # handle escaped quotes
    on_bad_lines='skip'   # skip problematic rows
)

In [10]:
print(listings_NL.columns)
print(reviews_NL.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [11]:
Combined_Amsterdam = pd.merge(
    listings_NL,
    reviews_NL,
    left_on='id',         # from listings
    right_on='listing_id',# from reviews
    how='left'
)

# Drop duplicate listing_id column from reviews
Combined_Amsterdam.drop(columns=['listing_id'], inplace=True)

# Save merged Amsterdam dataset
Combined_Amsterdam.to_csv("Amsterdam.csv", index=False)


In [12]:
Combined_Amsterdam.isnull().sum()

id_x                                                 0
listing_url                                          0
scrape_id                                            0
last_scraped                                         0
source                                               0
name                                                 0
description                                       9771
neighborhood_overview                            23007
picture_url                                          0
host_id                                              0
host_url                                             0
host_name                                            4
host_since                                           4
host_location                                     1489
host_about                                       18129
host_response_time                               21673
host_response_rate                               21673
host_acceptance_rate                             12846
host_is_su

In [13]:
# Create summary table
summary = pd.DataFrame({
    'Column': Combined_Amsterdam.columns,
    'Total_Rows': len(Combined_Amsterdam),
    'Unique_Values': [Combined_Amsterdam[col].nunique() for col in Combined_Amsterdam.columns],
    'Duplicate_Count': [Combined_Amsterdam[col].duplicated().sum() for col in Combined_Amsterdam.columns],
    'Duplicate_Percentage': [round((Combined_Amsterdam[col].duplicated().sum() / len(Combined_Amsterdam)) * 100, 2) for col in Combined_Amsterdam.columns]
})

# Display full table without truncation
pd.set_option('display.max_rows', None)
print(summary)

                                          Column  Total_Rows  Unique_Values  \
0                                           id_x      142566          10168   
1                                    listing_url      142566          10168   
2                                      scrape_id      142566              1   
3                                   last_scraped      142566              1   
4                                         source      142566              2   
5                                           name      142566           9885   
6                                    description      142566           9528   
7                          neighborhood_overview      142566           4882   
8                                    picture_url      142566          10078   
9                                        host_id      142566           8930   
10                                      host_url      142566           8930   
11                                     host_name    

In [14]:
# Load your already merged datasets
albany = pd.read_csv("Albany.csv")
amsterdam = pd.read_csv("Amsterdam.csv")

# Add a city column to each
albany["city"] = "Albany"
amsterdam["city"] = "Amsterdam"

# Combine into one DataFrame
Combined = pd.concat([albany, amsterdam], ignore_index=True)

# Save combined dataset
Combined.to_csv("Airbnb.csv", index=False)

print("Combined dataset saved as Airbnb.csv")


/tmp/ipykernel_664/2100425936.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  albany = pd.read_csv("Albany.csv")
/tmp/ipykernel_664/2100425936.py:3: DtypeWarning: Columns (0,80,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  amsterdam = pd.read_csv("Amsterdam.csv")


Combined dataset saved as Airbnb.csv


In [15]:
# Load your combined file
Combined = pd.read_csv("Airbnb.csv")

# If both id_x and id_y exist, keep listing_id and drop them
Combined = Combined.drop(columns=['id_x', 'id_y'], errors='ignore')

# Fill missing review-related fields with placeholders
Combined['comments'] = Combined['comments'].fillna("No review")
Combined['reviewer_name'] = Combined['reviewer_name'].fillna("Unknown")
Combined['reviewer_id'] = Combined['reviewer_id'].fillna(-1)
Combined['date'] = Combined['date'].fillna("No date")

# Optionally fill host_about with "No info"
Combined['host_about'] = Combined['host_about'].fillna("No host info")

print("Cities in Combined file:", Combined['city'].unique() if 'city' in Combined.columns else "No city column")

/tmp/ipykernel_664/1055489429.py:2: DtypeWarning: Columns (0,72,80,82,83,85) have mixed types. Specify dtype option on import or set low_memory=False.
  Combined = pd.read_csv("Airbnb.csv")


Cities in Combined file: ['Albany' 'Amsterdam']


In [16]:
print(Combined.columns.tolist())

['listing_id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'a

In [17]:
Combined.to_csv("Airbnb_Project.csv", index=False)
print("Cleaned dataset saved. Cities present:", Combined['city'].unique())

Cleaned dataset saved. Cities present: ['Albany' 'Amsterdam']


In [18]:
Airbnb_Project = pd.read_csv("Airbnb_Project.csv")
# 1. Clean numeric columns
for col in Airbnb_Project.columns:
    # Try converting to numeric (for cases like price, bathrooms_text, etc.)
    Airbnb_Project[col] = pd.to_numeric(Airbnb_Project[col], errors='ignore')
    
# Separate numeric & non-numeric
num_cols = Airbnb_Project.select_dtypes(include=['float64', 'int64']).columns
cat_cols = Airbnb_Project.select_dtypes(exclude=['float64', 'int64']).columns

# Fill missing values
Airbnb_Project[num_cols] = Airbnb_Project[num_cols].fillna(Airbnb_Project[num_cols].median())
Airbnb_Project[cat_cols] = Airbnb_Project[cat_cols].fillna("Unknown")

# Step 1: Remove currency symbols and convert to float
# Remove $ and commas, strip spaces
Airbnb_Project['price'] = Airbnb_Project['price'].replace('[\$,]', '', regex=True).str.strip()

# Replace "Unknown" or empty strings with NaN
Airbnb_Project['price'] = Airbnb_Project['price'].replace(['Unknown', ''], np.nan)

# Convert to float
Airbnb_Project['price'] = Airbnb_Project['price'].astype(float)

# Fill missing prices with median
Airbnb_Project['price'].fillna(Airbnb_Project['price'].median(), inplace=True)

print(Airbnb_Project['price'].head())

# Step 2: Fill missing prices with the median
Airbnb_Project['price'].fillna(Airbnb_Project['price'].median(), inplace=True)

# Step 3: Verify that no missing values remain
print(Airbnb_Project.isnull().sum()['price'])

# Step 4: Drop specific columns
Airbnb_Project.drop(columns=['neighbourhood_group_cleansed', 'calendar_updated'])

# Step 5: Save without overwriting the DataFrame variable
Airbnb_Project.to_csv("Airbnb_Cleaned.csv", index=False)

print("Missing prices filled & file saved as Airbnb_Cleaned.csv")

/tmp/ipykernel_664/2810781793.py:1: DtypeWarning: Columns (0,72) have mixed types. Specify dtype option on import or set low_memory=False.
  Airbnb_Project = pd.read_csv("Airbnb_Project.csv")


0    70.0
1    70.0
2    70.0
3    70.0
4    70.0
Name: price, dtype: float64
0
Missing prices filled & file saved as Airbnb_Cleaned.csv


In [19]:
AC = pd.read_csv("Airbnb_Cleaned.csv")
print(AC.isnull().sum())
print("Cleaned dataset saved. Cities present:", AC['city'].unique())

/tmp/ipykernel_664/738677296.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  AC = pd.read_csv("Airbnb_Cleaned.csv")


listing_id                                           0
listing_url                                          0
scrape_id                                            0
last_scraped                                         0
source                                               0
name                                                 0
description                                          0
neighborhood_overview                                0
picture_url                                          0
host_id                                              0
host_url                                             0
host_name                                            0
host_since                                           0
host_location                                        0
host_about                                           0
host_response_time                                   0
host_response_rate                                   0
host_acceptance_rate                                 0
host_is_su

In [20]:
# forecast average price by date
df = AC.groupby('date').agg({'price': 'mean'}).reset_index()

# Rename columns for Prophet
df = df.rename(columns={'date': 'ds', 'price': 'y'})

# Convert to datetime, forcing invalid ones to NaT
df['ds'] = pd.to_datetime(df['ds'], errors='coerce')

# Drop rows where date couldn't be parsed
df = df.dropna(subset=['ds'])


In [ ]:
# Filter data for Albany and Amsterdam
albany_df = AC[AC["city"] == "Albany"]
amsterdam_df = AC[AC["city"] == "Amsterdam"]

# Combine them into one DataFrame
reviews_df = pd.concat([albany_df, amsterdam_df], ignore_index=True)

print(reviews_df["city"].value_counts())

# Ensure comments column exists
if "comments" not in reviews_df.columns:
    raise ValueError("No 'comments' column found in reviews data.")

# Replace NaN comments with empty strings
reviews_df["comments"] = reviews_df["comments"].fillna("")

# Sentiment analysis
reviews_df["polarity"] = reviews_df["comments"].apply(lambda x: TextBlob(x).sentiment.polarity)
reviews_df["subjectivity"] = reviews_df["comments"].apply(lambda x: TextBlob(x).sentiment.subjectivity)

# Classify sentiment
def get_sentiment(score):
    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    else:
        return "Neutral"

reviews_df["sentiment"] = reviews_df["polarity"].apply(get_sentiment)

# Group by city & sentiment
sentiment_counts = reviews_df.groupby(["city", "sentiment"]).size().reset_index(name="count")

# Save results
reviews_df.to_csv("Airbnb_reviews_sentiment_by_city.csv", index=False)
sentiment_counts.to_csv("Sentiment_summary_by_city.csv", index=False)

# Print summary
print(sentiment_counts)


city
Amsterdam    146416
Albany        18921
Name: count, dtype: int64
